In [1]:
def seed_everything(seed = 0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

In [2]:
class param:
    num_epoch = 8
    batch_size = 64
    num_data = 12800
    img_size = 224
    img_face_crop_mergin = 60
    img_center_crop_size = 224
    num_workers = 4
    lr = 0.0001
    k_fold = 5
    train_patience = 4

class path:
    train_csv = '/opt/ml/input/data/train/train.csv'
    train_images = '/opt/ml/input/data/train/images'
    eval_images = '/opt/ml/input/data/eval/images'
    model_save = '/opt/ml/code/results'

class label:
    def age(x):
        x = int(x)
        assert(x > 0)
        return 0 if x < 30 else 1 if x < 60 else 2

    def gender(x):
        assert(x == 'male' or x == 'female')
        return 0 if x == 'male' else 1

    def mask(x):
        assert(x.startswith('incorrect') or x.startswith('mask') or x.startswith('normal'))
        return 0 if x.startswith('mask') else 1 if x.startswith('incorrect') else 2

    def group(x):
        x = os.path.basename(x).split('_')
        return label.gender(x[1]) * 3 + label.age(x[3])
    
    def image(x):
        x = os.path.split(x)
        mask_label = label.mask(x[-1])
        x = x[-2].split('_')
        age_label = label.age(x[3])
        gender_label = label.gender(x[1])
        return mask_label * 6 + gender_label * 3 + age_label

In [3]:
import os
import pickle
import torch
import random
import pickle
import timm
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from tqdm.notebook import tqdm
from glob import glob
from collections import defaultdict
from pprint import pprint
from PIL import Image
from albumentations import Compose, Resize, HorizontalFlip, Rotate
from adamp import AdamP


In [4]:
seed_everything()

class FaceDataset(Dataset):
    def __init__(self, imgs, transforms):
        self.imgs = imgs
        self.transforms = transforms
        self.labels = [label.image(img) for img in imgs]
        self.bboxes = [pickle.load(open(os.path.splitext(img)[0] + '.txt', 'rb')) for img in self.imgs]

    def __getitem__(self, idx):
        img = Image.open(self.imgs[idx])
        label = self.labels[idx]
        img_size = img.size
        box = self.bboxes[idx]
        if box is not None:
            margin = param.img_face_crop_mergin
            box = [
                int(max(box[0] - margin / 2, 0)),
                int(max(box[1] - margin / 2, 0)),
                int(min(box[2] + margin / 2, img_size[0])),
                int(min(box[3] + margin / 2, img_size[1]))
            ]
        else:
            margin = param.img_center_crop_size
            box = [
                int(img_size[0] / 2 - margin / 2),
                int(img_size[1] / 2 - margin / 2),
                int(img_size[0] / 2 + margin / 2),
                int(img_size[1] / 2 + margin / 2)
            ]
        img = img.crop(box)
        if self.transforms is not None:
            img = self.transforms(image=np.array(img))['image']
        return img, label
    
    def __len__(self):
        return len(self.imgs)

def get_dataset(images_path, transforms=None, k_fold = 6, num_data=2700):
    #num_data : 3 * k_fold * n
    groups = glob(images_path + '/*')

    #classifying by group label
    groups_classified = defaultdict(list)
    for group in groups:
        groups_classified[label.group(group)].append(group)
    groups_classified = list(groups_classified.values())

    #upsampling/downsampling to num_data
    num_data //= 3
    groups_flattened = []
    num_class = len(groups_classified)
    num_data_per_class = num_data // k_fold
    for group_classified in groups_classified:
        extra = random.sample(group_classified, num_data // k_fold % len(group_classified))
        group_classified *= num_data // k_fold // len(group_classified)
        group_classified += extra
        groups_flattened += group_classified
    
    #shuffle and divide
    random.shuffle(groups_flattened)
    groups_fold = [groups_flattened[i * num_data_per_class : (i + 1) * num_data_per_class] for i in range(k_fold)]

    #convert group path to image path
    imgs_fold = [[] for _ in range(k_fold)]
    for i, group_fold in enumerate(groups_fold):
        for group in group_fold:
            ext = [os.path.splitext(i)[1] for i in glob(group + '/*')]
            ext = '.jpg' if '.jpg' in ext else '.jpeg' if '.jpeg' in ext else '.png'
            imgs = ['incorrect_mask', 'mask' + str(random.randint(1, 5)), 'normal']
            imgs = [os.path.join(group, img) + ext for img in imgs]
            imgs_fold[i] += imgs

    #create dataset
    dataset_fold = []
    for i in range(k_fold):
        dataset_fold.append(FaceDataset(imgs_fold[i], transforms=transforms))

    return dataset_fold

transforms = Compose([
    Resize(param.img_size, param.img_size, p=1.0),
    HorizontalFlip(p=.5),
    Rotate(10)
])
    
dataset = get_dataset(path.train_images, transforms)

In [16]:
def train(name, model, train_data, valid_data, optimizer, scheduler, criterion):
    num_epoch = param.num_epoch
    train_log_interval = 5

    best_val_acc = 0
    best_val_loss = np.inf
    renew_count = 0

    device = torch.device('cuda')
    model.to(device)
    for epoch in tqdm(range(num_epoch)):
        model.train()
        loss_value = 0
        matches = 0
        for i, data in enumerate(train_data):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs.permute(0, 3, 1, 2).float())
            preds = torch.argmax(outputs, dim=-1)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()

            if (i + 1) % train_log_interval == 0:
                train_loss = loss_value / train_log_interval
                train_acc = matches / param.batch_size / train_log_interval
                print(f'{name}) epoch: {epoch + 1}, batch: {i + 1}, train loss: {train_loss:5.3}, train accuracy: {train_acc:5.3%}')

                loss_value = 0
                matches = 0
        
        model.eval()
        loss_value = 0
        matches = 0
        for i, data in enumerate(valid_data):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs.permute(0, 3, 1, 2).float())
                preds = torch.argmax(outputs, dim=-1)

                loss_value += criterion(outputs, labels).item()
                matches += (preds == labels).sum().item()
            
        val_loss = loss_value / len(valid_data)
        val_acc = matches / len(valid_data) / param.batch_size

        if val_loss < best_val_loss:
            best_val_loss = val_loss
        
        if val_acc > best_val_acc:
            print("New best model for f1-score! saving the model")
            save_path = path.model_save + '/' + name
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            torch.save(model.state_dict(), save_path + f'/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt')
            best_val_acc = val_acc
            renew_count = 0
        else:
            renew_count += 1
        
        if renew_count > param.train_patience:
            print("Early stopping")
            break

        print('val_acc:', val_acc, 'loss:', val_loss, 'best acc:', best_val_acc, 'best loss:', best_val_loss)
        scheduler.step(val_loss)

    print('Finishing Training')

In [15]:
for fold in tqdm(range(param.k_fold)):
    trainset = ConcatDataset([dataset[i] for i in range(param.k_fold) if i != fold])
    validset = dataset[fold]
    train_data = DataLoader(trainset, batch_size=param.batch_size, shuffle=True, num_workers=param.num_workers)
    valid_data = DataLoader(validset, batch_size=param.batch_size, shuffle=True, num_workers=param.num_workers)
    model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=18)
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr=param.lr)
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5)
    train(name=f'last_{fold}_{param.img_size}', model=model, train_data=train_data, valid_data=valid_data, optimizer=optimizer, scheduler=scheduler, criterion=criterion)

New best model for f1-score! saving the model
val_acc: 0.048828125 loss: 553.6003909111023 best acc: 0.048828125 best loss: 553.6003909111023
New best model for f1-score! saving the model
val_acc: 0.056640625 loss: 689.5002410411835 best acc: 0.056640625 best loss: 553.6003909111023
val_acc: 0.046875 loss: 551.7532777786255 best acc: 0.056640625 best loss: 551.7532777786255
Exception ignored in: <function _releaseLock at 0x7fe4282a88c0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt




RuntimeError: DataLoader worker (pid(s) 29521, 29522, 29523) exited unexpectedly